In [2]:
## Insert all the imports up here

import pandas as pd
import matplotlib.pyplot as plt
import numpy
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
import statsmodels.api as sm
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima_model import ARIMA

In C:\Users\souvi\anaconda3\envs\tsa_course\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The savefig.frameon rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.
In C:\Users\souvi\anaconda3\envs\tsa_course\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The verbose.level rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.
In C:\Users\souvi\anaconda3\envs\tsa_course\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The verbose.fileo rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.


In [3]:
## Import Travel Data

travelData = pd.read_csv('/home/souvik/Repos/Datasets/airline-passengers.csv', index_col=0, parse_dates=True)
travelData.head()

FileNotFoundError: File b'/home/souvik/Repos/Datasets/airline-passengers.csv' does not exist

In [ ]:
travelData.info()

In [ ]:
## Plot the data

travelData.plot(figsize=(10, 8))

In [ ]:
## Moving average of data

travelData['MA'] = travelData['Passengers'].rolling(3).mean()
travelData.plot(figsize=(10, 8))

In [ ]:
## Seasonal decomposition of data

s_decomp = seasonal_decompose(travelData['Passengers'])
result = s_decomp.plot()
result.set_size_inches((10, 8))

In [ ]:
## Apply Augmented Dickey-Fuller Test

adf = adfuller(travelData['Passengers'])
print('Adf statistic: %f' % adf[0])

## As adf > 0.05, data is non-stationary

In [ ]:
travelData['First Diff'] = travelData['Passengers'] - travelData['Passengers'].shift(1)
travelData['First Diff'].plot(figsize=(10,8))

In [ ]:
adf = adfuller(travelData['First Diff'].dropna())
print('Adf statistic: %f' % adf[0])

In [ ]:
acf = plot_acf(travelData['First Diff'].dropna())

In [ ]:
pacf = plot_pacf(travelData['First Diff'].dropna())

In [ ]:
travelData['Seasonal First Diff'] = travelData['Passengers'] - travelData['Passengers'].shift(12)
travelData['Seasonal First Diff'].plot()

In [ ]:
adf = adfuller(travelData['Seasonal First Diff'].dropna())
print('Adf statistic: %f' % adf[0])

In [ ]:
pacf = plot_pacf(travelData['Seasonal First Diff'].dropna())

In [ ]:
acf = plot_acf(travelData['Seasonal First Diff'].dropna())

In [ ]:
test_start, test_end = '1949-01-01', '1958-12-01'
exp_start, exp_end = '1959-01-01', '1960-12-01'

In [ ]:
test = travelData['Passengers'][test_start:test_end]

In [ ]:
exp = travelData['Passengers'][exp_start:exp_end]

In [ ]:
test.plot()

In [ ]:
exp.plot()

In [ ]:
# p = 1, d = 1, q = 1
# sp  = 1, d = 1, q = 5
model=sm.tsa.statespace.SARIMAX(test, order=(1,1,1), seasonal_order=(2,1,5,12))

In [ ]:
result = model.fit()

In [ ]:
result.resid.plot(kind='hist')

In [ ]:
prediction = result.predict(start=exp_start, end=exp_end+12)

In [ ]:
prediction.plot(figsize=(10, 8), label='Pred')
travelData['Passengers'].plot(label='Data')
plt.legend()